In [1]:
import numpy as np
from tqdm import trange
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import nn
from tqdm import tqdm

import mowl
mowl.init_jvm('10g')
from mowl.datasets import PathDataset, Dataset
from mowl.base_models import EmbeddingELModel
from mowl.nn import ELEmModule, ELBoxModule, BoxSquaredELModule
from mowl.utils.data import FastTensorDataLoader

from src.utils import *
from src.elmodule import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


In [3]:
experiments = [{'file_name' : 'family',
                'format_' : None,
                'add_noise': False},
               {'file_name' : 'family_noisy_gnn_100',
                'format_' : None,
                'add_noise': True},
               {'file_name' : 'family_noisy_gnn_1000',
                'format_' : None,
                'add_noise': True},
               #{'file_name' : 'family_noisy_gnn_10000',
               # 'format_' : None,
               # 'add_noise': True},
               #{'file_name' : 'family_noisy_gnn_100000',
               # 'format_' : None,
               # 'add_noise': True},
               {'file_name' : 'family_noisy_random_100',
                'format_' : None,
                'add_noise': True},
               {'file_name' : 'family_noisy_random_1000',
                'format_' : None,
                'add_noise': True}#,
               #{'file_name' : 'family_noisy_random_10000',
               # 'format_' : None,
               # 'add_noise': True},
               #{'file_name' : 'family_noisy_random_100000',
               # 'format_' : None,
               # 'add_noise': True}
              ]

**Models**

In [ ]:
elem = ElModel(dataset, 
               module_name='elem', 
               dim=200, 
               margin=0.1, 
               batch_size=4096*8, 
               test_batch_size=32, 
               epochs=500, 
               learning_rate=0.001,
               device=device)
elem._train()

In [ ]:
elbox = ElModel(dataset, 
                module_name='elbox', 
                dim=200, 
                margin=0.1, 
                batch_size=4096*8, 
                test_batch_size=32, 
                epochs=500, 
                learning_rate=0.001,
                device=device)
elbox._train()

**Eval**

In [ ]:
elem._eval('subsumption')

In [ ]:
elem._eval('membership')

In [ ]:
elbox._eval('subsumption')

In [ ]:
elbox._eval('membership')

**Experiments: Box2EL**

In [10]:
for experiment in experiments: 
    file_name = experiment['file_name']
    format_ = experiment['format_']
    add_noise = experiment['add_noise']

    train_graph, valid_graph, test_graph, test_membership_graph, test_subsumption_graph, test_link_prediction_graph = split_ontology(file_name=file_name, 
                                                                                                                                     format_=format_, 
                                                                                                                                     train_ratio=1, 
                                                                                                                                     add_noise=add_noise)
    dataset = PathDataset(ontology_path=f'datasets/bin/{file_name}_train.owl',
                          testing_path=f'datasets/bin/{file_name}_test.owl',
                          validation_path=f'datasets/bin/{file_name}_val.owl')
    
    train_ont = preprocess_ontology_el(dataset.ontology)
    test_ont = preprocess_ontology_el(dataset.testing)
    valid_ont = preprocess_ontology_el(dataset.validation)

    dataset = Dataset(train_ont, testing=test_ont, validation=valid_ont)
    
    box2el = ElModel(dataset, 
                 module_name='box2el', 
                 dim=200, 
                 margin=0.1, 
                 batch_size=4096*8, 
                 test_batch_size=32, 
                 epochs=500, 
                 learning_rate=0.001,
                 device=device)
    box2el._train()
    
    print('Membership:')
    box2el._eval('membership')
    print('Subsumption:')
    box2el._eval('subsumption')
    print()

Triplets found in family.owl: 5017
Train Triplets found: 5017
Valid Triplets found: 0
Test Triplets found: 3891
Test Triplets (Membership) found: 1721
Test Triplets (Subsumption) found: 116
Test Triplets (Link Prediction) found: 2054


INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(36 100) 31)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '100'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(113 42) 170)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '170'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(105 42) 182)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '182'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(97 42) 55)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '97'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(43 Object

Main DataLoader: abox
Epoch: 0, Training: EL loss: 2.9824, ABox loss: 0.8353
Epoch: 100, Training: EL loss: 2.1408, ABox loss: 0.1765
Epoch: 200, Training: EL loss: 1.6607, ABox loss: 0.0124
Epoch: 300, Training: EL loss: 1.3193, ABox loss: 0.0025
Epoch: 400, Training: EL loss: 1.1107, ABox loss: 0.0013
Membership:
Mean Rank: 22.730, Median Rank: 21.000, MRR: 0.210
Hits@5: 0.236, Hits@10: 0.284
Subsumption:
Mean Rank: 14.013, Median Rank: 10.000, MRR: 0.195
Hits@5: 0.400, Hits@10: 0.513

Triplets found in family.owl: 5017
Triplets found in family_noisy_gnn_100.owl: 800
Train Triplets found: 5817
Valid Triplets found: 0
Test Triplets found: 3891
Test Triplets (Membership) found: 1721
Test Triplets (Subsumption) found: 116
Test Triplets (Link Prediction) found: 2054


INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(36 100) 31)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '100'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(113 42) 170)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '170'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(105 42) 182)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '182'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(97 42) 55)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '97'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(43 Object

Main DataLoader: abox
Epoch: 0, Training: EL loss: 2.9864, ABox loss: 0.8352
Epoch: 100, Training: EL loss: 2.1484, ABox loss: 0.1647
Epoch: 200, Training: EL loss: 1.6936, ABox loss: 0.0134
Epoch: 300, Training: EL loss: 1.3272, ABox loss: 0.0029
Epoch: 400, Training: EL loss: 1.1380, ABox loss: 0.0013
Membership:
Mean Rank: 20.198, Median Rank: 18.000, MRR: 0.222
Hits@5: 0.273, Hits@10: 0.349
Subsumption:
Mean Rank: 15.987, Median Rank: 9.000, MRR: 0.193
Hits@5: 0.393, Hits@10: 0.547

Triplets found in family.owl: 5017
Triplets found in family_noisy_gnn_1000.owl: 7998
Train Triplets found: 13015
Valid Triplets found: 0
Test Triplets found: 3891
Test Triplets (Membership) found: 1721
Test Triplets (Subsumption) found: 116
Test Triplets (Link Prediction) found: 2054


...

Main DataLoader: abox
Epoch: 0, Training: EL loss: 2.9712, ABox loss: 0.8360
Epoch: 100, Training: EL loss: 2.1580, ABox loss: 0.1573
Epoch: 200, Training: EL loss: 1.6665, ABox loss: 0.0109
Epoch: 300, Training: EL loss: 1.3093, ABox loss: 0.0030
Epoch: 400, Training: EL loss: 1.1370, ABox loss: 0.0013
Membership:
Mean Rank: 22.649, Median Rank: 21.000, MRR: 0.209
Hits@5: 0.237, Hits@10: 0.343
Subsumption:
Mean Rank: 12.120, Median Rank: 7.000, MRR: 0.207
Hits@5: 0.433, Hits@10: 0.673

Triplets found in family.owl: 5017
Triplets found in family_noisy_random_100.owl: 783
Train Triplets found: 5800
Valid Triplets found: 0
Test Triplets found: 3891
Test Triplets (Membership) found: 1721
Test Triplets (Subsumption) found: 116
Test Triplets (Link Prediction) found: 2054


INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(36 100) 31)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '100'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(113 42) 170)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '170'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(105 42) 182)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '182'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(97 42) 55)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '97'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(43 Object

Main DataLoader: abox
Epoch: 0, Training: EL loss: 2.9917, ABox loss: 0.8384
Epoch: 100, Training: EL loss: 2.1501, ABox loss: 0.1675
Epoch: 200, Training: EL loss: 1.6796, ABox loss: 0.0173
Epoch: 300, Training: EL loss: 1.3528, ABox loss: 0.0025
Epoch: 400, Training: EL loss: 1.1092, ABox loss: 0.0015
Membership:
Mean Rank: 20.968, Median Rank: 20.000, MRR: 0.216
Hits@5: 0.247, Hits@10: 0.323
Subsumption:
Mean Rank: 12.773, Median Rank: 7.000, MRR: 0.202
Hits@5: 0.427, Hits@10: 0.593

Triplets found in family.owl: 5017
Triplets found in family_noisy_random_1000.owl: 7839
Train Triplets found: 12856
Valid Triplets found: 0
Test Triplets found: 3891
Test Triplets (Membership) found: 1721
Test Triplets (Subsumption) found: 116
Test Triplets (Link Prediction) found: 2054


INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(36 100) 31)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '100'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(113 42) 170)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '170'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(105 42) 182)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '182'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(97 42) 55)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '97'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(43 Object

Main DataLoader: abox
Epoch: 0, Training: EL loss: 2.9911, ABox loss: 0.8351
Epoch: 100, Training: EL loss: 2.1265, ABox loss: 0.1552
Epoch: 200, Training: EL loss: 1.6812, ABox loss: 0.0142
Epoch: 300, Training: EL loss: 1.3148, ABox loss: 0.0032
Epoch: 400, Training: EL loss: 1.1125, ABox loss: 0.0015
Membership:
Mean Rank: 21.266, Median Rank: 19.000, MRR: 0.216
Hits@5: 0.246, Hits@10: 0.329
Subsumption:
Mean Rank: 15.260, Median Rank: 9.500, MRR: 0.192
Hits@5: 0.380, Hits@10: 0.540

